# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
df = pd.read_csv('../WeatherPy/output_data/output_data_file.csv')
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-175.2000,82.40,89,75,9.22,TO,1612663856
1,Yellowknife,62.4560,-114.3525,-41.80,74,1,3.44,CA,1612663636
2,Altzayanca,19.4333,-97.7833,64.40,36,1,11.50,MX,1612664249
3,Connersville,39.6412,-85.1411,23.00,73,90,4.61,US,1612664241
4,Pisco,-13.7000,-76.2167,69.80,88,90,6.91,PE,1612663960
...,...,...,...,...,...,...,...,...,...
585,Camden,39.7835,-74.9663,36.00,79,100,1.23,US,1612664276
586,Amapá,1.0000,-52.0000,73.11,98,88,1.83,BR,1612664070
587,Chany,55.3121,76.7647,28.56,98,100,20.15,RU,1612664277
588,Montrose,38.4166,-108.2673,42.80,28,20,9.22,US,1612664277


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
locations = df[['Lat', 'Lng']]
humidity = df['Humidity']

# Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
hotel_df = df.loc[(df['Max Temp'] < 80) & (df['Max Temp'] >= 70)]
hotel_df = hotel_df.loc[df['Wind Speed'] < 10]
hotel_df = hotel_df.loc[df['Cloudiness'] == 0]
hotel_df = hotel_df.dropna()
hotel_df.reset_index(inplace=True, drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Te Anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612663916
1,Methala,10.2166,76.2166,77.00,94,0,2.30,IN,1612664253
2,Tzucacab,20.0667,-89.0500,78.80,69,0,4.70,MX,1612664263
3,Zárate,-34.0981,-59.0286,72.00,84,0,4.09,AR,1612664264
4,Harper,4.3750,-7.7169,79.02,88,0,8.81,LR,1612664005
5,Parrita,9.5201,-84.3225,73.40,83,0,2.30,CR,1612664266
6,General Roca,-39.0333,-67.5833,75.20,69,0,2.30,AR,1612664005
7,Neuquén,-38.9516,-68.0591,75.20,69,0,2.30,AR,1612663702
8,Lázaro Cárdenas,17.9583,-102.2000,73.83,75,0,1.74,MX,1612663810
9,Bambous Virieux,-20.3428,57.7575,75.00,86,0,1.99,MU,1612663862


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Te Anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612663916,
1,Methala,10.2166,76.2166,77.00,94,0,2.30,IN,1612664253,
2,Tzucacab,20.0667,-89.0500,78.80,69,0,4.70,MX,1612664263,
3,Zárate,-34.0981,-59.0286,72.00,84,0,4.09,AR,1612664264,
4,Harper,4.3750,-7.7169,79.02,88,0,8.81,LR,1612664005,


In [32]:
params = {
    'radius': 5000,
    'query': 'hotel',
    'key': g_key
}

print('Beginning Data Retrieval')
print('-----------------------------')

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    
    base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
    
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f'{lat}, {lng}'

    # Make request
    results = requests.get(base_url, params=params).json()['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print(f'Retrieving data for the #{index + 1} city: {city}...')
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

print('-----------------------------')
print('Data Retrieval Completed.')

hotel_df

Beginning Data Retrieval
-----------------------------
Retrieving data for the #1 city: Te Anau...
Retrieving data for the #2 city: Methala...
Retrieving data for the #3 city: Tzucacab...
Retrieving data for the #4 city: Zárate...
Retrieving data for the #5 city: Harper...
Retrieving data for the #6 city: Parrita...
Retrieving data for the #7 city: General Roca...
Retrieving data for the #8 city: Neuquén...
Retrieving data for the #9 city: Lázaro Cárdenas...
Retrieving data for the #10 city: Bambous Virieux...
Retrieving data for the #11 city: Surubim...
Retrieving data for the #12 city: Lingao...
Retrieving data for the #13 city: Presidencia Roque Sáenz Peña...
-----------------------------
Data Retrieval Completed.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Te Anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612663916,Lakefront Lodge Te Anau
1,Methala,10.2166,76.2166,77.00,94,0,2.30,IN,1612664253,Coconut Garden
2,Tzucacab,20.0667,-89.0500,78.80,69,0,4.70,MX,1612664263,Hotel D'Rodi&Eve
3,Zárate,-34.0981,-59.0286,72.00,84,0,4.09,AR,1612664264,Sofitel La Reserva Cardales
4,Harper,4.3750,-7.7169,79.02,88,0,8.81,LR,1612664005,Grands' Guesthouse
5,Parrita,9.5201,-84.3225,73.40,83,0,2.30,CR,1612664266,Hotel Makanda by The Sea
6,General Roca,-39.0333,-67.5833,75.20,69,0,2.30,AR,1612664005,Estepa Hotel
7,Neuquén,-38.9516,-68.0591,75.20,69,0,2.30,AR,1612663702,Hilton Garden Inn Neuquen
8,Lázaro Cárdenas,17.9583,-102.2000,73.83,75,0,1.74,MX,1612663810,Hotel Playa Azul
9,Bambous Virieux,-20.3428,57.7575,75.00,86,0,1.99,MU,1612663862,Shangri-La's Le Touessrok Resort & Spa


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))